In [ ]:
using Pkg; Pkg.activate(".")

In [ ]:
using LinearAlgebra
using Random
using Distributions

using Plots

Random.seed!(42)

In [ ]:
include("nbstyle.jl")
include("util.jl")

In [ ]:
l = @layout [
    a{0.6w} [b{0.5h}
             b{0.5h}  ]
]

$$
\newcommand{\N}{\mathcal{N}}
\renewcommand{\vec}[1]{\boldsymbol{#1}}          % vector
\newcommand{\mat}[1]{\boldsymbol{#1}}  
\newcommand{\g}{\mid}
$$

## Build the state-space model

for states and measurements
$$
\vec{x}_k = \begin{pmatrix}x_k\\y_k\\\dot{x}_k\\\dot{y}_k\end{pmatrix} \in \mathbb{R}^D \qquad \vec{y}_k = \begin{pmatrix}\hat{x}_k \\ \hat{y}_k\end{pmatrix}\in \mathbb{R}^d \qquad k = 1, \dots, T
$$

In [ ]:
d, D = 2, 4;

### Initial distribution

$$\boldsymbol{x}_0 \sim \N(\vec{\mu}_0, \mat{\Sigma}_0)$$
with
$$
\begin{align}
\vec\mu_0 &= \vec{0}_D\\
\mat\Sigma_0 &= 10^{-5}\cdot \mat{I}_D
\end{align}
$$

In [ ]:
μ₀ = zeros(D) 
Σ₀ = Matrix(1e-5 * I(D));

### Dynamics
$$\boldsymbol{x}_k \mid \boldsymbol{x}_{k-1} \sim \N(\mat{A}_{k-1}\vec{x}_{k-1} + \vec{b}_{k-1}, \mat{Q}_{k-1})$$
with
$$
\begin{align}
    \mat{A}_k = \mat{A} &=
    \begin{pmatrix}
    1 & 0 & dt & 0 \\
    0 & 1 & 0 & dt \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\
    \end{pmatrix}\\[2mm]
    \mat{Q}_k = \mat{Q} &=
    \begin{pmatrix}
    dt^3/3  &   0     &  dt^2/2  &     0\\
        0  &   dt^3/3     & 0   &   dt^2/2\\
     dt^2/2  &   0   &      dt   &      0\\
        0   &  dt^2/2   &   0     &    dt\\
    \end{pmatrix}
\end{align}
$$
and
$$
dt := \lvert t_{k+1} - t_k \rvert
$$

In [ ]:
dt = 0.1

A = [1 0 dt  0
     0 1  0 dt
     0 0  1  0
     0 0  0  1]

Q = [dt^3/3     0       dt^2/2       0
        0     dt^3/3      0      dt^2/2
     dt^2/2     0         dt         0
        0     dt^2/2      0         dt];

### Measurement model

$$\boldsymbol{y}_k \mid \boldsymbol{x}_k \sim \N(\mat{H}_k \vec{x}_k + \vec{c}_{k}, \mat{R}_k)$$
with
$$
\begin{align}
    \mat{H}_k = \mat{H} &=
    \begin{pmatrix}
    1 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    \end{pmatrix}\\[2mm]
    \mat{R}_k = \mat{R} &= 0.2^2 \cdot \mat{I}_d
\end{align}
$$

In [ ]:
H = Matrix(I(D))[1:d, :]
R = Matrix(0.2^2 * I(d));

## Simulate a trajectory and draw noisy observations

In [ ]:
ground_truth, observations = simulate_linear(A, Q, zeros(D), H, R, zeros(d), μ₀, Σ₀, 200, rng=MersenneTwister(99));
state_idcs = 1:length(ground_truth)
data_idcs = unique(rand(state_idcs[1:end-1], 50));

In [ ]:
left_side = scatter(
    [y[1] for y in observations[data_idcs]], 
    [y[2] for y in observations[data_idcs]]; 
    label="Measurements", 
    legend=:topright,
    xlabel="x-position",
    ylabel="y-position",
    data_args...
)
plot!(left_side,
    [y[1] for y in ground_truth], 
    [y[2] for y in ground_truth];
    label="True Location",
    gt_args...)


right_side1 = scatter(
    (dt:dt:length(state_idcs)*dt)[data_idcs], 
    [y[1] for y in observations[data_idcs]]; 
    label="",
    ylabel="x-position",
    xlabel="time",
    data_args...
)
plot!(right_side1,
    dt:dt:length(state_idcs)*dt, 
    [y[1] for y in ground_truth];
    label="",
    gt_args...
)

right_side2 = scatter(
    (dt:dt:length(state_idcs)*dt)[data_idcs], 
    [y[2] for y in observations[data_idcs]]; 
    label="",
    ylabel="y-position",
    xlabel="time",
    data_args...
)
plot!(right_side2,
    dt:dt:length(state_idcs)*dt, 
    [y[2] for y in ground_truth];
    label="",
    gt_args...
)

both_sides = plot(left_side, right_side1, right_side2, layout=l, size=(800, 400))

## Filtering — The Kalman Filter

### Prediction
$$
\begin{align}
    \vec{\mu}^-_k    & = \mat{A}_{k-1}\vec{\mu}_{k-1} + \vec{b}_{k-1}                      \\
    \mat{\Sigma}^-_k & = \mat{A}_{k-1}\mat{\Sigma}_{k-1}\mat{A}_{k-1}^\top + \mat{Q}_{k-1}
\end{align}
$$

In [ ]:
function kf_predict(μ, Σ, A, Q)
    μ⁻ = A * μ # + b
    Σ⁻ = A * Σ * A' + Q
    return μ⁻, Σ⁻
end

### Correction
$$
\begin{align*}
    \hat{\vec{y}}_k & = \mat{H}_k \vec{\mu}^-_k + \vec{c}_k                     &
    \mat{S}_k       & = \mat{H}_k\mat{\Sigma}^-_k\mat{H}_k^\top + \mat{R}_k       \\
    \mat{K}_k       & = \mat{\Sigma}^-_k\mat{H}_k^\top\mat{S}^{-1}_k              \\
    \vec{\mu}_k     & = \vec{\mu}^-_k + \mat{K}_k (\vec{y}_k - \hat{\vec{y}}_k) &
    \mat{\Sigma}_k  & = \mat{\Sigma}^-_k - \mat{K}_k\mat{S}_k\mat{K}_k^\top
\end{align*}
$$

In [ ]:
function kf_correct(μ⁻, Σ⁻, H, R, y)
    y_hat = H * μ⁻ # + c
    S = H * Σ⁻ * H' + R
    K = Σ⁻ * H' / Symmetric(S)
    μ = μ⁻ + K * (y - y_hat)
    Σ = Σ⁻ - K * S * K'
    return μ, Σ
end

In [ ]:
filter_estimate = [(μ₀, Σ₀)]
predicted_moments = []
for k in state_idcs[2:end]
    # start from last point
    μ, Σ = filter_estimate[end]
    # predict
    μ⁻, Σ⁻ = kf_predict(μ, Σ, A, Q)
    push!(predicted_moments, (μ⁻, Σ⁻))
    # if there's data: correct
    if k ∈ data_idcs
        push!(filter_estimate, kf_correct(μ⁻, Σ⁻, H, R, observations[k]))
    else
        push!(filter_estimate, (μ⁻, Σ⁻))
    end
end

In [ ]:
left_side = plot(
    [y[1] for (y, s) in filter_estimate], 
    [y[2] for (y, s) in filter_estimate];
    label="Filter Estimate",
    xlabel="x-position",
    ylabel="y-position",
    legend=:bottomleft,
    filter_estimate_args...
)
plot!(left_side,
    [y[1] + 2sqrt(s[1,1]) for (y, s) in filter_estimate], 
    [y[2] + 2sqrt(s[2,2]) for (y, s) in filter_estimate]; 
    label=nothing,
    filter_cred_interval_args...
)
plot!(left_side,
    [y[1] - 2sqrt(s[1,1]) for (y, s) in filter_estimate], 
    [y[2] - 2sqrt(s[2,2]) for (y, s) in filter_estimate]; 
    label=nothing,
    filter_cred_interval_args...
)
plot!(left_side,
    [y[1] for y in ground_truth], 
    [y[2] for y in ground_truth]; 
    label="True Location",
    gt_args...
)
scatter!(left_side,
    [y[1] for y in observations[data_idcs]], 
    [y[2] for y in observations[data_idcs]]; 
    label="Measurements",
    data_args...
)


right_side1 = plot(
    dt:dt:length(filter_estimate)*dt,
    [y[1] for (y, s) in filter_estimate],
    ribbon=[2s[1, 1] for (y, s) in filter_estimate];
    xlabel="time",
    ylabel="x-position",
    ylim=(-5, 80),
    label="",
    filter_estimate_args...
)
scatter!(right_side1,
    (dt:dt:length(filter_estimate)*dt)[data_idcs], 
    [y[1] for y in observations[data_idcs]]; 
    label="",
    data_args...
)
plot!(right_side1,
    dt:dt:length(filter_estimate)*dt, 
    [y[1] for y in ground_truth];
    label="",
    gt_args...
)

right_side2 = plot(
    dt:dt:length(filter_estimate)*dt,
    [y[2] for (y, s) in filter_estimate],
    ribbon=[2s[2, 2] for (y, s) in filter_estimate];
    xlabel="time",
    ylabel="y-position",
    label="",
    ylim=(-15,10),
    filter_estimate_args...
)
scatter!(right_side2,
    (dt:dt:length(filter_estimate)*dt)[data_idcs], 
    [y[2] for y in observations[data_idcs]]; 
    label="",
    data_args...
)
plot!(right_side2,
    dt:dt:length(filter_estimate)*dt, 
    [y[2] for y in ground_truth];
    label="",
    gt_args...
)

both_sides = plot(left_side, right_side1, right_side2, layout=l, size=(800, 400))

In [ ]:
filt_anim_parts = []
filt_anim = @animate for i in 1:length(filter_estimate)
    frame = plot(
        [y[1] for y in ground_truth], 
        [y[2] for y in ground_truth];
        label="True Location",
        xlabel="x-position",
        ylabel="y-position",
        xlim=(-5, 80),
        ylim=(-15,10),
        legend=:bottomleft,
        size=(800, 400),
        gt_args...
    )
    scatter!(
        frame, 
        [y[1] for y in observations[data_idcs]], 
        [y[2] for y in observations[data_idcs]]; 
        label="Measurements",
        data_args...
    )
    plot!(
        frame,
        [y[1] for (y, s) in filter_estimate[1:i]], 
        [y[2] for (y, s) in filter_estimate[1:i]];
        label="Filter Estimate",
        filter_estimate_args...
    )
    plot!(
        frame,
        [y[1] + 2sqrt(s[1,1]) for (y, s) in filter_estimate[1:i]], 
        [y[2] + 2sqrt(s[2,2]) for (y, s) in filter_estimate[1:i]]; 
        label=nothing,
        filter_cred_interval_args...
    )
    plot!(
        frame,
        [y[1] - 2sqrt(s[1,1]) for (y, s) in filter_estimate[1:i]], 
        [y[2] - 2sqrt(s[2,2]) for (y, s) in filter_estimate[1:i]]; 
        label=nothing,
        filter_cred_interval_args...
    )
    
    if i < length(filter_estimate)
        predicted_future = [filter_estimate[i]]
        for (j, _t) in enumerate(filter_estimate[i+1:end])
            m, S = predicted_future[end]
            push!(
                predicted_future,
                kf_predict(m, S, A, Q)
            )
        end
        plot!(
            frame,
            [y[1] for (y, s) in predicted_future],
            [y[2] for (y, s) in predicted_future]; 
            label="Predicted estimate",
            prediction_estimate_args...
        )
        plot!(
            frame,
            [y[1] + 2sqrt(s[1,1]) for (y, s) in predicted_future],
            [y[2] + 2sqrt(s[2,2]) for (y, s) in predicted_future];
            label=nothing,
            prediction_cred_interval_args...
        )
        plot!(
            frame,
            [y[1] - 2sqrt(s[1,1]) for (y, s) in predicted_future],
            [y[2] - 2sqrt(s[2,2]) for (y, s) in predicted_future];
            label=nothing,
            prediction_cred_interval_args...
        )
    end
    
    push!(filt_anim_parts, frame)
end
gif(filt_anim, "out/filt_car_tracking.gif", fps=20, loop=0)

## Smoothing — The Rauch-Tung-Striebel Smoother

$$
\begin{align}
    \mat{G}_{k} &= \mat{\Sigma}_{k}\mat{A}_{k}^\top \left[\mat{\Sigma}^-_{k+1}\right]^{-1} \\
    \vec{\xi}_k &= \vec{\mu}_k + \mat{G}_{k} \left(\vec{\xi}_{k+1} - \vec{\mu}^-_{k+1}\right)\\
    \mat{\Lambda}_{k} &= \mat{\Sigma}_{k} + \mat{G}_{k}\left(\mat{\Lambda}_{k+1} - \mat{\Sigma}^-_{k+1}\right)\mat{G}_k^\top
\end{align}
$$

In [ ]:
function kf_smooth(μ, Σ, ξ_next, Λ_next, μ⁻, Σ⁻, A, Q)
    G = Σ * A' / Symmetric(Σ⁻)
    ξ = μ + G * (ξ_next - μ⁻)
    Λ = Σ + G * (Λ_next - Σ⁻) * G'
    return ξ, Λ
end

In [ ]:
smoother_estimate = [filter_estimate[end]]
for k in reverse(state_idcs[1:end-1])
    ξ_next, Λ_next = smoother_estimate[1]
    μ, Σ = filter_estimate[k]
    μ⁻, Σ⁻ = predicted_moments[k]
    pushfirst!(smoother_estimate, kf_smooth(μ, Σ, ξ_next, Λ_next, μ⁻, Σ⁻, A, Q))
end

In [ ]:
filt2smooth_anim_parts = []
filt2smooth_anim = @animate for i in reverse(2:length(smoother_estimate)-1)
    frame = plot(
        [y[1] for y in ground_truth], 
        [y[2] for y in ground_truth]; 
        label="True Location",
        xlabel="x-position",
        ylabel="y-position",
        legend=:bottomleft,
        gt_args...
    )
    scatter!(
        frame, 
        [y[1] for y in observations[data_idcs]], 
        [y[2] for y in observations[data_idcs]]; 
        label="Measurements", 
        data_args...
    )
    
    plot!(
        frame,
        [y[1] for (y, s) in filter_estimate[1:i]], 
        [y[2] for (y, s) in filter_estimate[1:i]];
        label="Filter estimate",
        filter_estimate_args...
    )
    plot!(
        frame,
        [y[1] + 2sqrt(s[1,1]) for (y, s) in filter_estimate[1:i-1]],
        [y[2] + 2sqrt(s[2,2]) for (y, s) in filter_estimate[1:i-1]];
        label=nothing,
        filter_cred_interval_args...
    )
    plot!(
        frame,
        [y[1] - 2sqrt(s[1,1]) for (y, s) in filter_estimate[1:i-1]], 
        [y[2] - 2sqrt(s[2,2]) for (y, s) in filter_estimate[1:i-1]]; 
        label=nothing,
        filter_cred_interval_args...
    )
    plot!(
        frame,
        [y[1] + 2sqrt(s[1,1]) for (y, s) in pushfirst!(deepcopy(smoother_estimate[i:end]), filter_estimate[i-1])], 
        [y[2] + 2sqrt(s[2,2]) for (y, s) in pushfirst!(deepcopy(smoother_estimate[i:end]), filter_estimate[i-1])];
        label=nothing,
        smoother_cred_interval_args...
    )
    plot!(
        frame,
        [y[1] - 2sqrt(s[1,1]) for (y, s) in pushfirst!(deepcopy(smoother_estimate[i:end]), filter_estimate[i-1])], 
        [y[2] - 2sqrt(s[2,2]) for (y, s) in pushfirst!(deepcopy(smoother_estimate[i:end]), filter_estimate[i-1])]; 
        label=nothing,
        smoother_cred_interval_args...
    )
    plot!(
        frame,
        [y[1] for (y, s) in smoother_estimate[i:end]], 
        [y[2] for (y, s) in smoother_estimate[i:end]];
        label="Smoother estimate",
        smoother_estimate_args...
    )
    push!(filt2smooth_anim_parts, frame)
end
gif(filt2smooth_anim, "out/filt2smooth_anim_car_tracking.gif", fps=20)

In [ ]:
left_side = plot(
    [y[1] for (y, s) in smoother_estimate], 
    [y[2] for (y, s) in smoother_estimate];
    label="Smoother Estimate",
    xlabel="x-position",
    ylabel="y-position",
    legend=:bottomleft,
    smoother_estimate_args...
)
plot!(left_side,
    [y[1] + 2sqrt(s[1,1]) for (y, s) in smoother_estimate], 
    [y[2] + 2sqrt(s[2,2]) for (y, s) in smoother_estimate]; 
    label=nothing,
    smoother_cred_interval_args...
)
plot!(left_side,
    [y[1] - 2sqrt(s[1,1]) for (y, s) in smoother_estimate], 
    [y[2] - 2sqrt(s[2,2]) for (y, s) in smoother_estimate]; 
    label=nothing,
    smoother_cred_interval_args...
)
plot!(left_side,
    [y[1] for y in ground_truth], 
    [y[2] for y in ground_truth]; 
    label="True Location",
    gt_args...
)
scatter!(left_side,
    [y[1] for y in observations[data_idcs]], 
    [y[2] for y in observations[data_idcs]]; 
    label="Measurements",
    data_args...
)


right_side1 = plot(
    dt:dt:length(smoother_estimate)*dt,
    [y[1] for (y, s) in smoother_estimate],
    ribbon=[2s[1, 1] for (y, s) in smoother_estimate];
    xlabel="time",
    ylabel="x-position",
    ylim=(-5, 80),
    label="",
    smoother_estimate_args...
)
scatter!(right_side1,
    (dt:dt:length(smoother_estimate)*dt)[data_idcs], 
    [y[1] for y in observations[data_idcs]]; 
    label="",
    data_args...
)
plot!(right_side1,
    dt:dt:length(smoother_estimate)*dt, 
    [y[1] for y in ground_truth];
    label="",
    gt_args...
)

right_side2 = plot(
    dt:dt:length(smoother_estimate)*dt,
    [y[2] for (y, s) in smoother_estimate],
    ribbon=[2s[2, 2] for (y, s) in smoother_estimate];
    xlabel="time",
    ylabel="y-position",
    label="",
    ylim=(-15,10),
    smoother_estimate_args...
)
scatter!(right_side2,
    (dt:dt:length(smoother_estimate)*dt)[data_idcs], 
    [y[2] for y in observations[data_idcs]]; 
    label="",
    data_args...
)
plot!(right_side2,
    dt:dt:length(smoother_estimate)*dt, 
    [y[2] for y in ground_truth];
    label="",
    gt_args...
)

plot(left_side, right_side1, right_side2, layout=l, size=(800, 400))